In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
movies = pd.read_csv('movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [3]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
movies.shape

(3883, 3)

In [5]:
users = pd.read_csv('users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')


In [6]:
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [7]:
ratings.shape

(387848, 4)

In [8]:
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [9]:
#Preparing training and testing tests
training_set = pd.read_csv('u1.base', delimiter = '\t')
training_set.head()

,1,1.1,5,874965758
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,5,3,889751712
4,1,7,4,875071561


In [10]:
training_set = np.array(training_set, dtype = 'int')

In [11]:
test_set = pd.read_csv('u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

In [12]:
training_set.shape

(79999, 4)

In [13]:
#Getting number of users and movies

nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0])) )
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])) )

In [14]:
#convert as users in lines and movies in columns
#lists within lists

def convert(data):
  new_data = []
  for id_users in range(1, nb_users+1): #+1 to include all users upto 943
    id_movies = data[:, 1][data[:, 0] == id_users]
    id_ratings = data[:, 2][data[:, 0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[ id_movies - 1 ] = id_ratings
    new_data.append(list(ratings))
  return new_data

training_set = convert(training_set)
test_set = convert(test_set)

In [15]:
print(len(test_set))

943


In [16]:
#converting the data into torch sensors

training_set = torch.FloatTensor(training_set) #torch sensors multidimension array of same dtype
test_set = torch.FloatTensor(test_set) #floattensor requires list of lists

In [17]:
len(training_set)

943

In [18]:
#converting ratings into binary 0/1
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

In [19]:
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    def train(self, v0, vk, ph0, phk):
        self.W += torch.mm(v0.t(), ph0).t() - torch.mm(vk.t(), phk).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

In [20]:
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(0.3425)
epoch: 2 loss: tensor(0.2506)
epoch: 3 loss: tensor(0.2298)
epoch: 4 loss: tensor(0.2365)
epoch: 5 loss: tensor(0.2462)
epoch: 6 loss: tensor(0.2487)
epoch: 7 loss: tensor(0.2494)
epoch: 8 loss: tensor(0.2489)
epoch: 9 loss: tensor(0.2486)
epoch: 10 loss: tensor(0.2466)


In [21]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
        v = training_set[id_user:id_user+1]
        vt = test_set[id_user:id_user+1] #target
        if len(vt[vt>=0]) > 0:
            _,h = rbm.sample_h(v)
            _,v = rbm.sample_v(h)

        test_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]) + 1e-8)  # Avoid division by zero

        s += 1.
print(' loss: '+str(test_loss/s))

 loss: tensor(0.2534)


In [22]:
_, predictions = rbm.sample_v(hk)
original_ratings = (v0 >= 0)
predicted_rating_values_already_rated = predictions[original_ratings]
predictions[v0 >= 0] = v0[v0 >= 0]

print("Original Ratings that the model predicted :", predicted_rating_values_already_rated)
print("Final Ratings :", predictions[v0 >= 0])
print("New Predicted Ratings (for missing values):", predictions[v0 < 0])


Original Ratings that the model predicted : tensor([1., 1., 1.,  ..., 1., 1., 1.])
Final Ratings : tensor([1., 1., 1.,  ..., 0., 0., 0.])
New Predicted Ratings (for missing values): tensor([1., 1., 0.,  ..., 1., 1., 1.])


In [23]:
recommended_movies = torch.argsort(predictions[v0 < 0], descending=True)[:5]
print(recommended_movies)


tensor([95260, 95283, 95281, 95279, 95278])
